# L01: Introduction & Linear Regression

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/methods-algorithms/blob/master/notebooks/L01_linear_regression.ipynb)

**Course**: Methods and Algorithms - MSc Data Science

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the OLS framework and derive the normal equation
2. Implement linear regression from scratch using NumPy
3. Apply gradient descent for optimization
4. Interpret coefficients in a business context

## Setup

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Set random seed for reproducibility
np.random.seed(42)

# Plotting settings
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print('Setup complete!')

## 1. Generate Synthetic Housing Data

We create a synthetic housing dataset for hands-on practice.

In [ ]:
# Load housing dataset (with fallback to synthetic generation)
import os

# Try to load the actual dataset first
try:
    # Try local path first
    if os.path.exists('../datasets/housing_synthetic.csv'):
        df = pd.read_csv('../datasets/housing_synthetic.csv')
        print('Loaded dataset from local file')
    else:
        # Try GitHub URL for Colab
        url = 'https://raw.githubusercontent.com/Digital-AI-Finance/methods-algorithms/master/datasets/housing_synthetic.csv'
        df = pd.read_csv(url)
        print('Loaded dataset from GitHub')
except Exception as e:
    print(f'Could not load dataset: {e}')
    print('Generating synthetic data instead...')
    
    # Generate synthetic housing data (fallback)
    n_samples = 200

    # Features
    size = np.random.uniform(50, 250, n_samples)  # sqm
    bedrooms = np.random.randint(1, 6, n_samples)  # number of bedrooms
    age = np.random.uniform(0, 50, n_samples)  # years
    distance_center = np.random.uniform(1, 20, n_samples)  # km to city center

    # True coefficients
    true_intercept = 50000
    true_coefs = [2000, 15000, -800, -3000]  # size, bedrooms, age, distance

    # Generate target with noise
    noise = np.random.normal(0, 30000, n_samples)
    price = (true_intercept + 
             true_coefs[0] * size + 
             true_coefs[1] * bedrooms + 
             true_coefs[2] * age + 
             true_coefs[3] * distance_center + 
             noise)

    # Create DataFrame
    df = pd.DataFrame({
        'size_sqm': size,
        'bedrooms': bedrooms,
        'age_years': age,
        'distance_km': distance_center,
        'price': price
    })

print(f'Dataset shape: {df.shape}')
df.head()

In [ ]:
# Visualize the data
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].scatter(df['size_sqm'], df['price']/1000, alpha=0.5)
axes[0, 0].set_xlabel('Size (sqm)')
axes[0, 0].set_ylabel('Price (thousands)')
axes[0, 0].set_title('Price vs Size')

axes[0, 1].scatter(df['bedrooms'], df['price']/1000, alpha=0.5)
axes[0, 1].set_xlabel('Bedrooms')
axes[0, 1].set_ylabel('Price (thousands)')
axes[0, 1].set_title('Price vs Bedrooms')

axes[1, 0].scatter(df['age_years'], df['price']/1000, alpha=0.5)
axes[1, 0].set_xlabel('Age (years)')
axes[1, 0].set_ylabel('Price (thousands)')
axes[1, 0].set_title('Price vs Age')

axes[1, 1].scatter(df['distance_km'], df['price']/1000, alpha=0.5)
axes[1, 1].set_xlabel('Distance to Center (km)')
axes[1, 1].set_ylabel('Price (thousands)')
axes[1, 1].set_title('Price vs Distance')

plt.tight_layout()
plt.show()

## 2. Theory: The Normal Equation

For linear regression, the optimal coefficients minimize the sum of squared errors:

$$\hat{\beta} = (X^T X)^{-1} X^T y$$

## 3. Implementation from Scratch

In [ ]:
# Prepare data
X = df[['size_sqm', 'bedrooms', 'age_years', 'distance_km']].values
y = df['price'].values

# Add intercept column (column of 1s)
X_with_intercept = np.column_stack([np.ones(len(X)), X])

print(f'X shape: {X_with_intercept.shape}')
print(f'y shape: {y.shape}')

In [ ]:
# Normal equation implementation
def fit_normal_equation(X, y):
    """Fit linear regression using the normal equation."""
    # beta = (X'X)^{-1} X'y
    XtX = X.T @ X
    Xty = X.T @ y
    beta = np.linalg.solve(XtX, Xty)  # More stable than inv()
    return beta

# Fit model
beta_normal = fit_normal_equation(X_with_intercept, y)

print('Coefficients from Normal Equation:')
print(f'  Intercept: {beta_normal[0]:.2f}')
print(f'  Size:      {beta_normal[1]:.2f} per sqm')
print(f'  Bedrooms:  {beta_normal[2]:.2f} per bedroom')
print(f'  Age:       {beta_normal[3]:.2f} per year')
print(f'  Distance:  {beta_normal[4]:.2f} per km')

In [ ]:
# Compare with true coefficients
print('\nComparison with true coefficients:')
print(f'  Intercept: True={true_intercept}, Estimated={beta_normal[0]:.0f}')
for i, name in enumerate(['Size', 'Bedrooms', 'Age', 'Distance']):
    print(f'  {name}: True={true_coefs[i]}, Estimated={beta_normal[i+1]:.0f}')

## 4. Gradient Descent Implementation

In [ ]:
def fit_gradient_descent(X, y, learning_rate=0.0001, n_iterations=1000, tol=1e-6):
    """Fit linear regression using gradient descent.
    
    Args:
        X: Design matrix with intercept column
        y: Target values
        learning_rate: Step size for gradient updates
        n_iterations: Maximum iterations
        tol: Convergence tolerance - stops when loss change < tol
        
    Returns:
        beta: Fitted coefficients
        losses: Loss history for each iteration
    """
    n, p = X.shape
    beta = np.zeros(p)  # Initialize coefficients
    losses = []
    
    for i in range(n_iterations):
        # Predictions
        y_pred = X @ beta
        
        # Compute loss
        loss = np.mean((y - y_pred)**2)
        losses.append(loss)
        
        # Check convergence criterion
        if i > 0 and abs(losses[-2] - losses[-1]) < tol:
            print(f'Converged at iteration {i} (loss change < {tol})')
            break
        
        # Compute gradient
        gradient = -2/n * X.T @ (y - y_pred)
        
        # Update coefficients
        beta = beta - learning_rate * gradient
    
    return beta, losses

# Normalize features for gradient descent (important!)
X_normalized = (X - X.mean(axis=0)) / X.std(axis=0)
X_norm_with_intercept = np.column_stack([np.ones(len(X)), X_normalized])

# Fit model with convergence criterion
beta_gd, losses = fit_gradient_descent(X_norm_with_intercept, y, 
                                        learning_rate=0.1, 
                                        n_iterations=1000,
                                        tol=1e-6)

print(f'Total iterations: {len(losses)}')
print(f'Final loss: {losses[-1]:.2f}')

In [ ]:
# Plot convergence
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Mean Squared Error')
plt.title('Gradient Descent Convergence')
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()

## 5. Using scikit-learn

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predictions
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

# Evaluate
print('scikit-learn LinearRegression:')
print(f'  Train R2: {r2_score(y_train, y_pred_train):.4f}')
print(f'  Test R2:  {r2_score(y_test, y_pred_test):.4f}')
print(f'  Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}')

In [ ]:
# Feature importance (coefficients)
feature_names = ['Size (sqm)', 'Bedrooms', 'Age (years)', 'Distance (km)']

plt.figure(figsize=(10, 5))
plt.barh(feature_names, lr.coef_)
plt.xlabel('Coefficient Value')
plt.title('Feature Coefficients')
plt.axvline(x=0, color='black', linewidth=0.5)
plt.show()

print('\nCoefficient interpretation:')
for name, coef in zip(feature_names, lr.coef_):
    direction = 'increases' if coef > 0 else 'decreases'
    print(f'  - 1 unit increase in {name} {direction} price by ${abs(coef):.0f}')

## 6. Residual Analysis

In [ ]:
# Residuals
residuals = y_test - y_pred_test

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals vs Fitted
axes[0].scatter(y_pred_test/1000, residuals/1000, alpha=0.5)
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_xlabel('Fitted Values (thousands)')
axes[0].set_ylabel('Residuals (thousands)')
axes[0].set_title('Residuals vs Fitted Values')

# Q-Q plot
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot of Residuals')

plt.tight_layout()
plt.show()

## 7. Regularization: Ridge and Lasso

In [ ]:
# Compare OLS, Ridge, and Lasso
from sklearn.preprocessing import StandardScaler

# Standardize features for fair comparison
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit models
ols = LinearRegression()
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=100)

ols.fit(X_train_scaled, y_train)
ridge.fit(X_train_scaled, y_train)
lasso.fit(X_train_scaled, y_train)

# Compare coefficients
print('Standardized Coefficients Comparison:')
print(f'{"Feature":<15} {"OLS":>12} {"Ridge":>12} {"Lasso":>12}')
print('-' * 55)
for i, name in enumerate(feature_names):
    print(f'{name:<15} {ols.coef_[i]:>12.0f} {ridge.coef_[i]:>12.0f} {lasso.coef_[i]:>12.0f}')

## Exercises

### Exercise 1: Implement RMSE
Write a function to compute RMSE from scratch.

In [ ]:
# SOLUTION: Implement RMSE from scratch
def rmse(y_true, y_pred):
    """Compute Root Mean Squared Error.

    RMSE measures the average magnitude of prediction errors,
    penalizing larger errors more heavily due to squaring.

    Args:
        y_true: Actual target values
        y_pred: Predicted values

    Returns:
        float: RMSE value (same units as target)
    """
    # Step 1: Compute squared errors
    squared_errors = (y_true - y_pred) ** 2
    # Step 2: Take mean of squared errors (MSE)
    mse = np.mean(squared_errors)
    # Step 3: Take square root to get RMSE
    return np.sqrt(mse)

# Test the implementation
y_pred_test = lr.predict(X_test)
our_rmse = rmse(y_test, y_pred_test)
sklearn_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'Our RMSE: {our_rmse:.2f}')
print(f'sklearn RMSE: {sklearn_rmse:.2f}')
print(f'Match: {np.isclose(our_rmse, sklearn_rmse)}')

### Exercise 2: Cross-Validation
Use 5-fold cross-validation to find the optimal Ridge alpha.

In [ ]:
# SOLUTION: Cross-Validation for Ridge alpha selection
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

# Define alpha values to test (log scale is common)
alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

# Store results for comparison
results = []

print('Ridge Regularization - 5-Fold Cross-Validation')
print('=' * 50)
print(f'{"Alpha":<12} {"Mean R2":<12} {"Std R2":<12}')
print('-' * 50)

best_alpha = None
best_score = -np.inf

for alpha in alphas:
    # Create Ridge model with current alpha
    ridge = Ridge(alpha=alpha)

    # Perform 5-fold CV, scoring with R2
    cv_scores = cross_val_score(ridge, X_train, y_train, cv=5, scoring='r2')

    mean_score = cv_scores.mean()
    std_score = cv_scores.std()
    results.append({'alpha': alpha, 'mean': mean_score, 'std': std_score})

    print(f'{alpha:<12.3f} {mean_score:<12.4f} {std_score:<12.4f}')

    # Track best alpha
    if mean_score > best_score:
        best_score = mean_score
        best_alpha = alpha

print('-' * 50)
print(f'Best alpha: {best_alpha} with CV R2: {best_score:.4f}')

# Final model with best alpha
final_ridge = Ridge(alpha=best_alpha)
final_ridge.fit(X_train, y_train)
test_r2 = r2_score(y_test, final_ridge.predict(X_test))
print(f'Test R2 with best alpha: {test_r2:.4f}')

## Summary

Key takeaways from this notebook:

1. Linear regression finds coefficients that minimize squared errors
2. Normal equation gives closed-form solution; gradient descent is iterative
3. Coefficients are directly interpretable as marginal effects
4. Regularization (Ridge/Lasso) prevents overfitting